# deep CNN

In [1]:
import os
os.environ['KERAS_BACKEND']= 'tensorflow'
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D,BatchNormalization
from keras.callbacks import ModelCheckpoint
model = Sequential()


model.add(Conv1D(32, 3, activation='relu',strides = 1, input_shape=(640,1)))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(3))
# model.add(Dropout(0.25))
model.add(BatchNormalization(axis=1))

model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
# model.add(Dropout(0.25))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
# model.add(Dropout(0.25))
model.add(BatchNormalization(axis=1))
model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(128,activation='relu'))
model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
model.summary()
# model.fit(x_train, y_train, batch_size=16, epochs=10)
# score = model.evaluate(x_test, y_test, batch_size=16)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 638, 32)           128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 636, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 212, 32)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 212, 32)           848       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 210, 64)           6208      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 208, 64)           12352     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 69, 64)            0         
__________

# Stacked LSTM

In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dense,Conv1D,Bidirectional,BatchNormalization
import numpy as np
import keras.backend as K
K.clear_session()

model = Sequential()
model.add(LSTM(32, return_sequences=True,input_shape=(20,32)))  
model.add(LSTM(32, return_sequences=True))  
model.add(LSTM(32))  

model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 32)            8320      
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 32)            8320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
Total params: 25,488
Trainable params: 25,488
Non-trainable params: 0
_________________________________________________________________


# Stacked GRU

In [5]:
from keras.models import Sequential
from keras.layers import GRU,Dense,Conv1D,Dropout
import keras.backend as K
K.clear_session()
import numpy as np
model = Sequential()
model.add(GRU(32, return_sequences=True, input_shape=(20,32)))  # returns a sequence of vectors of dimension 32
model.add(GRU(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(GRU(32))  # return a single vector of dimension 32
model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 20, 32)            6240      
_________________________________________________________________
gru_2 (GRU)                  (None, 20, 32)            6240      
_________________________________________________________________
gru_3 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
Total params: 19,248
Trainable params: 19,248
Non-trainable params: 0
_________________________________________________________________


# ResNet50

In [6]:
import keras
import keras_resnet.models
shape, classes = (26, 26, 1), 16  ####adjust the input dim and class number

x = keras.layers.Input(shape)

model = keras_resnet.models.ResNet50(x, classes=classes)
model.compile("nadam", "categorical_crossentropy", ["accuracy"])

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 26, 26, 1)    0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, 32, 32, 1)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 13, 13, 64)   3136        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 13, 13, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

# SAE

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
import keras.backend as K
K.clear_session()
input_img = Input(shape=(640,))
encoded_1 = Dense(128, activation='relu')(input_img)
encoded_2 = Dense(64, activation='relu')(encoded_1)
encoded_3 = Dense(32, activation='relu',activity_regularizer=regularizers.l1(10e-5))(encoded_2)#稀疏化这个自编码器
encoded_2 = Dense(64, activation='relu')(encoded_1)
encoded = Dense(128, activation='relu')(encoded_2)
decoded = Dense(640, activation='sigmoid')(encoded)
autoencoder = Model(input=input_img, output=decoded)
sm = Dense(16,activation='softmax')(encoded)
smer = Model(input=input_img, output=sm)
autoencoder.compile(optimizer='nadam', loss='binary_crossentropy')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 640)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               82048     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 640)               82560     
Total params: 181,184
Trainable params: 181,184
Non-trainable params: 0
_________________________________________________________________


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  from ipykernel import kernelapp as app
